# 认识Bedrock

![](images/bedrock_models.png)

### boto3创建 `bedrock` client

In [30]:
import json
import boto3
import base64



#Create the connection to Bedrock
bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-west-2', 
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2', 
    
)

### 列出可用模型版本

In [31]:
# List all foundation models
all_llms = [ model['modelId'] for model in bedrock.list_foundation_models()['modelSummaries']]
all_llms

['amazon.titan-tg1-large',
 'amazon.titan-embed-g1-text-02',
 'amazon.titan-text-lite-v1:0:4k',
 'amazon.titan-text-lite-v1',
 'amazon.titan-text-express-v1:0:8k',
 'amazon.titan-text-express-v1',
 'amazon.titan-embed-text-v1:2:8k',
 'amazon.titan-embed-text-v1',
 'amazon.titan-embed-image-v1:0',
 'amazon.titan-embed-image-v1',
 'amazon.titan-image-generator-v1:0',
 'amazon.titan-image-generator-v1',
 'stability.stable-diffusion-xl',
 'stability.stable-diffusion-xl-v0',
 'stability.stable-diffusion-xl-v1:0',
 'stability.stable-diffusion-xl-v1',
 'ai21.j2-grande-instruct',
 'ai21.j2-jumbo-instruct',
 'ai21.j2-mid',
 'ai21.j2-mid-v1',
 'ai21.j2-ultra',
 'ai21.j2-ultra-v1',
 'anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic.claude-3-sonnet-20240229-v1:0',
 'cohere.command-text-v14:7:4k',


### `Anthropic (Claude v3 (Sonnet))`

In [32]:
prompt_data = """What L in LLM means"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

In [33]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)

In [34]:
response_body = json.loads(response.get("body").read())
for output in response_body.get("content", []):
    print(output["text"])

In the context of LLM, the "L" stands for "Large". So LLM means "Large Language Model".

Large Language Models are neural network models that have been trained on vast amounts of text data to understand and generate human-like text. They are "large" in the sense that they have a huge number of parameters (sometimes in the billions) which allows them to capture extremely complex language patterns and knowledge.

Some well-known examples of LLMs include:

- GPT-3 (Generative Pre-trained Transformer 3) by OpenAI
- PaLM by Google 
- Jurassic-1 by AI21 Labs
- BLOOM by Hugging Face
- OPT by Meta AI
- Claude (the model powering me) by Anthropic

The term contrasts with smaller, more specialized language models used for tasks like sentiment analysis or named entity recognition. LLMs are general-purpose models that can handle a wide variety of natural language tasks after being prompted appropriately during inference time.


### With `Mistral (mixtral-8x7b)`

In [35]:
prompt_data = """<s>[INST]Craft a Python function to convert Celsius to Fahrenheit. If water boils at 100°C, what's that in Fahrenheit?[/INST]"""

body = json.dumps({ 
    'prompt': prompt_data,
    'max_tokens': 200,
    'top_p': 0.9,
    'temperature': 0.2,
})


modelId = 'mistral.mixtral-8x7b-instruct-v0:1'
accept = 'application/json'
contentType = 'application/json'

In [36]:
response = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                        modelId=modelId, 
                                        accept=accept, 
                                        contentType=contentType)

In [37]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('outputs')[0].get('text'))

 Here is a simple Python function to convert Celsius to Fahrenheit:

```python
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32
```

Now, let's use this function to convert 100 degrees Celsius to Fahrenheit:

```python
print(celsius_to_fahrenheit(100))
```

When you run this code, it will output `212.0`, which is the boiling point of water in Fahrenheit.


### 多模态图片输入 `Anthropic Claude v3 Models`

In [38]:
with open("images/cat.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')

payload = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": base64_string
                        }
                    },
                    {
                        "type": "text",
                        "text": "Write me a detailed description of this photo."
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body_bytes,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)


In [39]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('content')[0].get('text'))

The image depicts a cute and whimsical illustration of a cat. The cat is drawn in a cartoon-like style with exaggerated and expressive features. Its body is orange and white with distinct tiger stripes running along its back, tail, and legs. The cat's face has large, round eyes with amber-colored irises that give it a friendly and endearing look. Its mouth is open in a cheerful grin, revealing a pink tongue and small white teeth.

The cat's ears are pointed upwards, giving it an alert and lively expression. Its whiskers are prominent, adding to its feline charm. The illustration captures the cat's paws in a seated position, with its front paws resting on the ground and its tail curled neatly behind it.

The background is a solid gray color, which allows the vibrant colors and intricate details of the cat illustration to stand out. Overall, the image conveys a sense of joy and playfulness, capturing the endearing qualities of a beloved feline companion.
